# 02. Data Cleaning Notebook

## Objectives
- Remove duplicate entries from the dataset to ensure data quality
- Handle missing values using appropriate strategies
- Standardize data formats and fix inconsistent data types

## Inputs
- A cleaned version of the raw dataset file from data collection phase (i.e lang_proficiency_results)

## Outputs
- Cleaned dataset with consistent formatting and data types
- Documentation of data cleaning decisions and transformations applied
- Summary statistics of data quality improvements

## Additional information
- All cleaning operations are documented with clear rationale for reproducibility
- Any data removed or significantly modified is logged for transparency
- Raw data backups are preserved and data quality metrics tracked throughout the process
- Missing values and outliers are handled using domain-appropriate strategies with clear documentation

---

# Project Directory Structure

## Change working directory

We need to change the working directory from its current folder to the folder the code of this project is currently located

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\husse\\OneDrive\\Projects\\lang-level-pred\\jupyter_notebooks'

In [2]:
from pathlib import Path

# swtich to project root directory
project_root = Path.cwd().parent
os.chdir(project_root)
print(f"Working directory: {os.getcwd()}")

Working directory: c:\Users\husse\OneDrive\Projects\lang-level-pred


---

# Data loading and basic exploration
This code block imports fundamental Python libraries for data analysis and visualization and checks their versions

- pandas: For data manipulation and analysis
- numpy: For numerical computations
- matplotlib: For creating visualizations and plots

The version checks help ensure:
- Code compatibility across different environments
- Reproducibility of analysis
- Easy debugging of version-specific issues

In [4]:
# Import data analysis tools
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


print(f"pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"matplotlib version: {matplotlib.__version__}")

pandas version: 2.3.1
NumPy version: 2.3.1
matplotlib version: 3.10.5


### List Files and Folders
- This code shows what files and folders are in our data folder and what folder we are currently in. 
- Subsequently we aren't in the right folder so the current path is set to the parent folder.

In [5]:
import os
from pathlib import Path

dataset_dir = Path("data/raw")
print(f"[INFO] Files/folders available in {dataset_dir}:")
os.listdir(dataset_dir)

[INFO] Files/folders available in data\raw:


['lang_proficiency_results_raw.csv']

## Load dataset
This code loads the dataset that is then displayed in the dataframe.

In [26]:
import pandas as pd
from pathlib import Path

# Define the path to the CSV file
file_path = Path("data/raw/lang_proficiency_results_raw.csv")

# Read the CSV file
df = pd.read_csv(file_path)

## Identifying problems in dataset

### Check Missing Values in Dataset
This code analyzes and displays missing values in our bulldozer dataset:
- Uses pandas' isna() function to identify missing values
- Counts total missing values per column using sum()
- Sorts results in descending order to highlight columns with most missing data

Previously this showed multiple missing values in some of the columns but i mistakenly ran it again after fixing the issue.

In [27]:
# Check missing values
df.isna().sum().sort_values(ascending=False)

overall_cefr       16
listening_score    16
speaking_score     14
reading_score       9
writing_score       8
user_id             0
dtype: int64

### Check for Duplicates
This code checks if any duplicates exist in the dataset

In [28]:
# Check if there are any duplicate rows
duplicates = df.duplicated()

# Count total duplicates
print(f"Number of duplicate rows: {duplicates.sum()}")

# View the duplicate rows (if any)
df[duplicates]

Number of duplicate rows: 10


,user_id,speaking_score,reading_score,listening_score,writing_score,overall_cefr
1000,508,32.0,31.0,35.0,39.0,A1
1001,819,57.0,55.0,58.0,56.0,B1
1002,453,61.0,59.0,57.0,70.0,B1
1003,369,53.0,55.0,47.0,53.0,A2
1004,243,51.0,47.0,41.0,48.0,A2
1005,930,82.0,72.0,71.0,74.0,B2
1006,263,89.0,91.0,90.0,88.0,C1
1007,811,29.0,40.0,27.0,31.0,A1
1008,319,52.0,50.0,50.0,47.0,A2
1009,50,27.0,32.0,40.0,25.0,A1


## Check for outliers
This code checks the dataset to see if any scores exist outide of the range 100 < x < 0

In [29]:
# Check outliers for all score columns
score_columns = ['speaking_score', 'listening_score', 'reading_score', 'writing_score']

for col in score_columns:
   outliers = ((df[col] < 1) | (df[col] > 100)).sum()
   print(f"{col}: {outliers} outliers")

speaking_score: 0 outliers
listening_score: 0 outliers
reading_score: 7 outliers
writing_score: 0 outliers


In [30]:
df.dtypes

user_id              int64
speaking_score     float64
reading_score      float64
listening_score    float64
writing_score      float64
overall_cefr        object
dtype: object

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          1010 non-null   int64  
 1   speaking_score   996 non-null    float64
 2   reading_score    1001 non-null   float64
 3   listening_score  994 non-null    float64
 4   writing_score    1002 non-null   float64
 5   overall_cefr     994 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 47.5+ KB


In [32]:
# Check if any non-numeric values exist
df['listening_score'].dtype == 'object'  # True if column contains strings

False

In [33]:
# Check if any non-numeric values exist
df['speaking_score'].dtype == 'object'  # True if column contains strings

False

In [34]:
# Check if any non-numeric values exist
df['writing_score'].dtype == 'object'  # True if column contains strings

False

In [35]:
# Check if any non-numeric values exist
df['reading_score'].dtype == 'object'  # True if column contains strings

False

## Data Cleaning

### Filling Missing CEFR Levels

- Some rows in the dataset are missing their overall_cefr value.
- The CEFR level is determined by the user’s skill scores (Speaking, Listening, Reading, Writing) 
- We can infer the missing CEFRs by calculating the average score per row and mapping it to the correct level.

We use the scoring thresholds defined in this dataset:

| Average Score | CEFR |
| ------------- | ---- |
| 20–40         | A1   |
| 41–55         | A2   |
| 56–70         | B1   |
| 71–85         | B2   |
| 86–92         | C1   |
| 93+           | C2   |


Example:

- If a user has scores (Speaking=72, Reading=74, Listening=70, Writing=71), the row average = 71.75 → this falls in the 71–85 range, so their overall_cefr is assigned as B2.

In [41]:
# Function to map average score → CEFR level
def assign_cefr(score):
    if score <= 40: return "A1"
    elif score <= 55: return "A2"
    elif score <= 70: return "B1"
    elif score <= 85: return "B2"
    elif score <= 92: return "C1"
    else: return "C2"

# Identify rows with missing CEFR
mask = df["overall_cefr"].isna()

# Compute row averages for those rows (ignoring NaNs in skills)
row_means = df.loc[mask, ["speaking_score", "reading_score", "listening_score", "writing_score"]].mean(axis=1, skipna=True)

# Assign CEFR levels
df.loc[mask, "overall_cefr"] = row_means.apply(assign_cefr)

#### Verify fix
- This code verifies there are no remaining mising values in the overall_cefr column.

In [42]:
# Check missing values
df.isna().sum().sort_values(ascending=False)

listening_score    16
speaking_score     14
reading_score       9
writing_score       8
user_id             0
overall_cefr        0
dtype: int64

### Handling Missing Values by Group Mean Imputation

In this dataset, each user’s skill scores (Speaking, Listening, Reading, Writing) directly influence their overall CEFR level.
If we replaced missing values with a global mean across all CEFR levels, it would distort the relationship between individual skills and the CEFR classification.

Instead, we fill missing values using the mean score within the same CEFR group.
This way, the imputation reflects the performance range typical of each level.

Example:

- If a speaking_score is missing for a B2 user:
- Collect all B2 users’ speaking scores.
- Compute the mean of those scores.
- Fill the missing value with this mean.
- This preserves the internal consistency of the dataset while ensuring no missing values remain.


In [43]:
# Fill missing values by CEFR group mean
for col in ["speaking_score", "reading_score", "listening_score", "writing_score"]:
    df[col] = df.groupby("overall_cefr")[col].transform(lambda x: x.fillna(x.mean()))

### Verify fix
- This code verifies the missing values where successfully addressed and should show no remaining missing values in any cell.

In [44]:
# Check missing values
df.isna().sum().sort_values(ascending=False)

user_id            0
speaking_score     0
reading_score      0
listening_score    0
writing_score      0
overall_cefr       0
dtype: int64

## Fixing Inconsistent Categorical Labels

In the dataset, some overall_cefr values appear in lowercase (e.g., "b2", "c2") while most are uppercase ("B2", "C2").
This inconsistency could cause issues when grouping or filtering by CEFR levels.

To standardize the labels, we convert all values in the overall_cefr column to uppercase:


In [47]:
# Capitalize all CEFR values to ensure consistency
df['overall_cefr'] = df['overall_cefr'].str.upper()

## Round Scores to Integers
- Since all scores are percentages, they should be stored as whole numbers.
- We round any decimal values to the nearest integer and cast the columns to integer type.
- This maintains consistency in the dataset and ensures that all scores are expressed as whole percentage values between 0 and 100.

In [48]:
df[['speaking_score', 'reading_score', 'listening_score', 'writing_score']] = \
df[['speaking_score', 'reading_score', 'listening_score', 'writing_score']].round(0).astype(int)

### Verify fix 
- This code gives a brief look at the updated dataset with the now rounded values

In [49]:
df.head(60)

,user_id,speaking_score,reading_score,listening_score,writing_score,overall_cefr
0,1,26,40,28,33,A1
1,2,91,92,89,87,C1
2,3,61,66,66,57,B1
3,4,65,60,55,55,B1
4,5,77,76,83,78,B2
5,6,58,66,61,56,B1
6,7,50,46,55,52,A2
7,8,78,71,82,81,B2
8,9,45,51,42,49,A2
9,10,64,64,68,56,B1
